In [21]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [22]:

# Evaluation Metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score

In [23]:
SEED = 1234

In [28]:
# # Load original DataSet and contact_df
# from google.colab import drive
# drive.mount('/content/drive') #,force_remount=Truimport os 
# cwd = os.getcwd()
# dataset_dir = os.path.join(cwd, 'DMTMDataset')
# # train_df=pd.read_csv(os.path.join(dataset_dir,'train.csv'))
# dataset_direction='/content/drive/MyDrive/'
# # !unzip ('/content/drive/MyDrive/')
# train_df=open(os.path.join(dataset_direction,'train.csv'))
# original_df=pd.read_csv((train_df))

# PCA_origin=open(os.path.join(dataset_direction,'PCA_origin.csv'))igin.csv'))

original_df=pd.read_csv('train.csv')
concat_df=pd.read_csv('k-best.csv')

# from numpy import genfromtxt
# PCA_arr = genfromtxt('PCA_origin.csv', delimiter=',')
# train_pca=pd.DataFrame(PCA_arr)

In [29]:
concat_df.head

<bound method NDFrame.head of         Unnamed: 0  CELL_TYPE_Macro  CELL_TYPE_Tx site  CELL_TYPE_micro  \
0                0                1                  0                0   
1                1                1                  0                0   
2                2                1                  0                0   
3                3                1                  0                0   
4                4                1                  0                0   
...            ...              ...                ...              ...   
621295      621295                1                  0                0   
621296      621296                1                  0                0   
621297      621297                1                  0                0   
621298      621298                1                  0                0   
621299      621299                1                  0                0   

        GEOGRAPHIC_CLUSTER_K_0  GEOGRAPHIC_CLUSTER_K_1  \
0          

In [30]:
train_data = pd.concat([original_df.loc[:,'SITE_ID'],concat_df.loc[:,:]], axis=1)
train_data.head()
# X = train_pca.copy()
# y = original_df[['SITE_ID','aircon_sum_target_next14d']]

,SITE_ID,Unnamed: 0,CELL_TYPE_Macro,CELL_TYPE_Tx site,CELL_TYPE_micro,GEOGRAPHIC_CLUSTER_K_0,GEOGRAPHIC_CLUSTER_K_1,GEOGRAPHIC_CLUSTER_K_3,GEOGRAPHIC_CLUSTER_K_4,GEOGRAPHIC_CLUSTER_K_5,...,ind_equipment_min_persistance_prev7d,ind_equipment_mean_persistance_prev7d,ind_min_rain_mm_prev7d,ind_mean_rain_mm_prev7d,ind_min_temperature_prev7d,aggregated_equipment_sum_alarms_prev3d,aggregated_fire/smoke_sum_alarms_prev3d,aggregated_ge_sum_alarms_prev3d,aggregated_power_sum_alarms_prev3d,aggregated_temperature_sum_alarms_prev3d
0,146,0,1,0,0,0,0,0,0,0,...,-0.088116,-0.230962,1.299733,0.818311,-0.656787,-0.298669,-0.204796,-0.319806,-0.272422,-0.331059
1,146,1,1,0,0,0,0,0,0,0,...,-0.088116,-0.230962,1.299733,0.891479,-0.586317,-0.298669,-0.204796,-0.319806,-0.272422,-0.331059
2,146,2,1,0,0,0,0,0,0,0,...,-0.088116,-0.230962,1.299733,1.268330,-0.586317,-0.298669,-0.204796,-0.319806,-0.272422,-0.331059
3,146,3,1,0,0,0,0,0,0,0,...,-0.088116,-0.230962,1.507274,1.254544,-0.663915,-0.298669,-0.204796,-0.319806,-0.272422,-0.331059
4,146,4,1,0,0,0,0,0,0,0,...,-0.088116,-0.230962,2.170650,1.280906,-0.900717,-0.298669,-0.204796,-0.319806,-0.272422,-0.331059


In [33]:
X=train_data.copy()
Y=original_df[['SITE_ID','aircon_sum_target_next14d']]

In [34]:
Site_IDs=X['SITE_ID'].unique().astype(int)
nSite_IDs=len(Site_IDs)
print(nSite_IDs)

2071


In [35]:
model= RandomForestClassifier()
evaluations=0
i=0
y_pred=[]
for site_id in Site_IDs:
    if i%100==0:
        print("Eval: "+str(i)+" of "+str(nSite_IDs)+" completed")
    
    #Making sub-dataset
    y_Site_ID=y[y['SITE_ID']==site_id].drop('SITE_ID',axis=1)
    X_Site_ID=X[X['SITE_ID']==site_id]
    X_train, X_test, y_train, y_test = train_test_split(X_Site_ID, y_Site_ID, test_size=0.2, random_state=SEED)
    
    #fitting the forest
    model.fit(X_train, np.ravel(y_train))
    y_pred.append(model.predict(X_test))

    X_test['aircon_sum_target_next14d'] = y_test

    #Saving the dataset for evaluation
    if(i==0):
        X_test_history=X_test.copy()
        y_test_history=y_test.copy()
    else:
        X_test_history=pd.concat([X_test_history,X_test], axis=0, ignore_index=True)
        y_test_history=pd.concat([y_test_history,y_test], axis=0, ignore_index=True)
    i+=1
y_pred_history=np.concatenate(y_pred, axis=0)
print(i)

Eval: 0 of 2071 completed
Eval: 100 of 2071 completed
Eval: 200 of 2071 completed
Eval: 300 of 2071 completed
Eval: 400 of 2071 completed
Eval: 500 of 2071 completed
Eval: 600 of 2071 completed
Eval: 700 of 2071 completed
Eval: 800 of 2071 completed
Eval: 900 of 2071 completed
Eval: 1000 of 2071 completed
Eval: 1100 of 2071 completed
Eval: 1200 of 2071 completed
Eval: 1300 of 2071 completed
Eval: 1400 of 2071 completed
Eval: 1500 of 2071 completed
Eval: 1600 of 2071 completed
Eval: 1700 of 2071 completed
Eval: 1800 of 2071 completed
Eval: 1900 of 2071 completed
Eval: 2000 of 2071 completed
2071


In [36]:
print("R2:",r2_score(y_test_history, y_pred_history))
print("\tAccuracy (Test) %.3f"%precision_score(y_test_history, y_pred_history))
print("\t      Precision %.3f"%precision_score(y_test_history, y_pred_history))
print("\t      Recall    %.3f"%recall_score(y_test_history, y_pred_history))
print("\t      F1        %.3f"%f1_score(y_test_history, y_pred_history))
print("\n")

R2: 0.7919283097999599
	Accuracy (Test) 0.939
	      Precision 0.939
	      Recall    0.848
	      F1        0.891




Having the model parameters and best fit model to our Train.csv Data, we will be able to predict over the Test.csv.
To be noticed, same scaled fitted model (standard Scalar) which has already implemented over the Train Data should be dedicated
toward the Test Test.

In [83]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import datetime

In [84]:
# from numpy import genfromtxt
# PCA_arr = genfromtxt('PCA_test.csv', delimiter=',')

In [ ]:
original_df=pd.read_csv('train.csv')
concat_df=pd.read_csv('k-best.csv')

In [85]:
test_origin = pd.read_csv('test.csv')

Having the model parameters and best fit model to our Train.csv Data, we will be able to predict over the Test.csv.
To be noticed, same scaled fitted model (standard Scalar) which has already implemented over the Train Data should be dedicated
toward the Test Test.

In [86]:
X_pca_test=pd.DataFrame(PCA_arr)
test_origin = test_origin[['SITE_ID','DATE','N_TRANSPORTED_SITES']]
# we should do all the preprocessing that we did for train.csv on the test.csv as well, The result is in the X_2 data frame

In [87]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.638963,-0.723192,1.073850,-0.020673,0.173960,-0.986872,-0.044315,-0.073762,0.318885,0.127459,...,0.228251,0.009626,0.024679,0.137854,0.068709,-0.048554,0.006412,0.025030,-0.178321,0.071160
1,-0.643231,-0.908548,1.280059,-0.098624,0.204755,-0.929311,-0.024817,-0.090516,0.362124,0.168404,...,0.249787,-0.011441,0.019608,0.166461,0.087784,-0.069482,-0.005775,0.014163,0.119573,0.014776
2,-0.630286,-1.024725,1.449132,-0.108238,0.159373,-0.975164,-0.014220,-0.097865,0.362856,0.169781,...,0.255645,-0.002174,0.017529,0.154766,0.077265,-0.049503,-0.007906,0.015206,0.299571,-0.017298
3,-0.615162,-0.894503,1.380044,-0.074874,0.003987,-1.386995,-0.097885,-0.032074,0.442236,0.228798,...,0.290155,-0.037637,0.012435,0.195928,0.110355,-0.091269,-0.026307,0.020561,0.489645,-0.063044
4,-0.622776,-1.300381,1.845811,-0.191194,0.013886,-1.246558,-0.029926,-0.086013,0.468155,0.261157,...,0.319756,-0.042134,-0.003155,0.240175,0.125228,-0.111572,-0.042119,0.016869,0.930811,-0.140841


In [88]:
test_origin.head()

,SITE_ID,DATE,N_TRANSPORTED_SITES
0,858,2019-04-10,10.0
1,858,2019-04-11,10.0
2,858,2019-04-12,10.0
3,858,2019-04-13,10.0
4,858,2019-04-14,10.0


In [89]:
test_df_concat=pd.concat([test_origin, X_pca_test], axis = 1)

/home/tara/anaconda3/envs/thesis/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.016777777777777777
